In [ ]:
from openai import OpenAI
import os 
import json 
import sys 
import requests
sys.path.append(r"C:\Users\o\agentflightbooking\fiancialAIAgent")

In [80]:
from openai import OpenAI
deepseek_client = OpenAI(
    api_key="sk-3da1e2817a2942ea8809277679bb05ec",
    base_url="https://api.deepseek.com/v1"
)

In [109]:
def reformuler_question_en_requete_usa(question, client):
    prompt = f"""
You are a legal research assistant for U.S. law.

Your job is to turn a user's natural language question into a focused Google search query
that finds relevant legal documents in PDF format on official U.S. government websites.
and as output give me just the query without explanations

Instructions:
- Use only the key legal terms (state, topic, law name if known).
- End the query with: site:.gov filetype:pdf
- Be concise and relevant.
- give just one query 

Example:
Question: What is the law about privacy rights in California?
Query: california privacy law site:.gov filetype:pdf

---

User question:
\"{question}\"

Google search query:
"""

    response = client.chat.completions.create(
        model="deepseek-reasoner",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )

    return response.choices[0].message.content.strip()


In [110]:
question = 'Can a person who travels to Cuba enter the US?'
reforulation = reformuler_question_en_requete_usa(question=question, client=deepseek_client)

In [111]:
reforulation

'US entry after Cuba travel regulations site:.gov filetype:pdf'

In [112]:
import os
import requests

SERPAPI_API_KEY = os.getenv("SERA_API_KEY")

def chercher_pdfs_via_serpapi(query, api_key=SERPAPI_API_KEY):
    url = "https://serpapi.com/search"
    params = {
        "engine": "google",
        "q": query,
        "api_key": api_key,
        "num": 5, 
        "filetype": "pdf",
        "hl": "en"
    }
    response = requests.get(url, params=params)
    if response.status_code != 200:
        raise Exception(f"Erreur SerpAPI : {response.status_code} - {response.text}")

    data = response.json()
    results = data.get("organic_results", [])
    pdf_urls = []
    for result in results:
        link = result.get("link", "")
        if link.lower().endswith(".pdf"):
            pdf_urls.append(link)
    return pdf_urls


In [113]:
pdf_link = chercher_pdfs_via_serpapi(query=reforulation)

In [114]:
pdf_link

['https://www.cbp.gov/sites/default/files/2025-06/CLP%20Bulletin%20%20Presidential%20Proclamation%20Restricting%20Entry%20Final.pdf',
 'https://travel.state.gov/content/dam/visas/iv-dv-supplemental/HAV%20-%20Havana-5-20-2024.pdf',
 'https://www.congress.gov/crs_external_products/IF/PDF/IF10045/IF10045.67.pdf']

In [115]:
import os
import requests
from datetime import datetime

def telecharger_pdf_dossier_temp(url_pdf, dossier_temp="data/temp"):
    os.makedirs(dossier_temp, exist_ok=True)
    
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S_%f")
    nom_fichier = f"document_{timestamp}.pdf"
    chemin_complet = os.path.join(dossier_temp, nom_fichier)
    
    response = requests.get(url_pdf)
    if response.status_code != 200:
        raise Exception(f"Erreur téléchargement PDF : {response.status_code}")
    
    with open(chemin_complet, "wb") as f:
        f.write(response.content)
    
    return chemin_complet


In [116]:
save_pdf = telecharger_pdf_dossier_temp(url_pdf=pdf_link[0])

In [117]:
save_pdf

'data/temp\\document_20250719_235609_502844.pdf'

In [118]:
from pdf2image import convert_from_bytes
from typing import List, Optional
from PIL import Image
import os
import requests
import json
import pytesseract

def ocr(pdf):
    images = convert_from_bytes(pdf)
    ocr_texts = []
    
    for i, image in enumerate(images):
        image_path = f"page_{i + 1}.png"
        image.save(image_path, "PNG")
        print(f"Saved {image_path}")
    images_path = [f"page_{i + 1}.png" for i in range(len(images))]
    for image_path in images_path:
        image = Image.open(image_path)
        with Image.open(image_path) as image:
                if image.mode != 'RGB':
                    image = image.convert('RGB')
                
                text = pytesseract.image_to_string(
                    image, 
                    lang='eng',
                    config='--psm 6'  
                )   
                text = text.replace('\n', ' ').strip()
                ocr_texts.append(text)
        os.remove(image_path)
        print(f"Removed {image_path}")
        
    return ocr_texts

In [119]:
#read pdf 
with open(save_pdf, "rb") as f:
    pdf_bytes = f.read()

In [120]:
ocr_texts = ocr(pdf_bytes)

Saved page_1.png
Saved page_2.png
Removed page_1.png
Removed page_2.png


In [93]:
ocr_texts

['Trusted Traveler Programs Handbook Office of Field Operations Admissibility and Passenger Programs Trusted Traveler Programs HB 3300-011 September 2022 Official Use Only RF U.S. Customs and Sus; Border Protection CBP-OFO-2022-092322-000004',
 'CBP-OFO-2022-092322-000005',
 'Foreword This handbook establishes the policies and procedures for the Trusted Traveler Programs (TTP). These guidelines include the processes for enrollment of travelers into the programs and the procedures for inspecting TTP members at the ports of entry. These programs are an essential element of the U.S. Customs and Border Protection’s (CBP) strategy to secure our borders through increased advanced knowledge of the travelers. TTP participants are subject to rigorous vetting and random compliance checks to ensure the integrity of the programs. Shorter wait times and expedited procedures for Trusted Travelers at the ports of entry provide the incentive for participation in the programs. The handbook provides an 

In [121]:
def ai_agent_analyze_ocr(ocr_texts, question,client):

    full_text = "\n\n".join(ocr_texts)
    
    prompt = f"""
You are a legal assistant helping to analyze PDF documents extracted by OCR.

User question: {question}

Below is the extracted text from the PDF document (from OCR):

{full_text}

Please provide a clear and concise answer to the user's question based on the content of the document.
"""
    
    response = client.chat.completions.create(
        model="deepseek-chat",
        messages=[
            {"role": "system", "content": "You are a helpful legal assistant."},
            {"role": "user", "content": prompt}
        ],
        temperature=0.2,
        max_tokens=800
    )
    
    answer = response.choices[0].message.content.strip()
    return answer

In [122]:
final_response = ai_agent_analyze_ocr(ocr_texts,question,client=deepseek_client)

In [123]:
print(final_response)

Based on the extracted text from the Presidential Proclamation dated June 4, 2025, **a person who travels to Cuba (a national of Cuba) will generally not be permitted to enter the United States** unless they fall under one of the exceptions listed.  

### Key Points:  
1. **General Restriction**: Cuba is one of the 19 countries whose nationals are subject to entry suspension under the Proclamation, effective **June 9, 2025**.  
2. **Exceptions**: A Cuban national may still enter the U.S. if they:  
   - Hold a **valid U.S. visa issued on or before June 8, 2025**.  
   - Are a **lawful permanent resident (LPR)** of the U.S.  
   - Are a **dual national traveling on a passport from a non-listed country**.  
   - Qualify for one of the other exceptions (e.g., certain visa classifications, family-based immigrant visas, etc.).  

### Action Required:  
If the traveler does not meet an exception, they **will not be allowed entry** unless the Proclamation is amended or revoked. For specific c

In [ ]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def semantic_chunking(text, model, threshold=0.5):
    sentences = text.split('.')
    embeddings = model.encode(sentences)
    
    similarities = [np.dot(embeddings[i], embeddings[i+1]) 
                   for i in range(len(embeddings)-1)]
    
    chunks = []
    current_chunk = sentences[0]
    
    for i, sim in enumerate(similarities):
        if sim < threshold:  
            chunks.append(current_chunk)
            current_chunk = sentences[i+1]
        else:
            current_chunk += '. ' + sentences[i+1]
    
    chunks.append(current_chunk)

    return chunks

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

c:\Python311\Lib\site-packages\huggingface_hub\file_download.py:143: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\o\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [69]:
from sentence_transformers import SentenceTransformer
import numpy as np

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

def semantic_chunking(text, model, threshold=0.5, min_chunk_len=3):

    sentences = [s.strip() for s in text.split('.') if s.strip()]
    if len(sentences) <= 1:
        return sentences  

    embeddings = model.encode(sentences, normalize_embeddings=True)

    similarities = [np.dot(embeddings[i], embeddings[i+1]) for i in range(len(embeddings) - 1)]

    chunks = []
    current_chunk = [sentences[0]]

    for i, sim in enumerate(similarities):
        if sim < threshold and len(current_chunk) >= min_chunk_len:
            chunks.append('. '.join(current_chunk) + '.')
            current_chunk = [sentences[i + 1]]
        else:
            current_chunk.append(sentences[i + 1])

    if current_chunk:
        chunks.append('. '.join(current_chunk) + '.')

    return chunks


In [124]:
ocr_text = "\n".join(ocr_texts)
chunks = semantic_chunking(ocr_text, model, threshold=0.5)
chunk_embeddings = model.encode(chunks, normalize_embeddings=True)

In [125]:
chunk_embeddings

array([[-0.02451354,  0.08217824, -0.0069642 , ..., -0.03024538,
        -0.02166775, -0.02374367],
       [ 0.00524226,  0.07700112, -0.01096253, ...,  0.0440694 ,
        -0.03081658,  0.0042694 ],
       [ 0.03243458,  0.01106589, -0.0193145 , ...,  0.01641932,
        -0.02276189,  0.0576993 ],
       ...,
       [ 0.05010303,  0.00984699, -0.00612123, ...,  0.03566615,
        -0.0437218 ,  0.03555981],
       [-0.00726368,  0.06085514,  0.00835826, ...,  0.00073932,
        -0.079758  ,  0.0094218 ],
       [ 0.02267454, -0.06435157, -0.00904555, ..., -0.00564285,
        -0.06875694, -0.05279756]], dtype=float32)

In [126]:
question_embeddings = model.encode(question,normalize_embeddings=True)

In [127]:
question_embeddings

array([-2.46259887e-02,  7.49730915e-02, -2.43164245e-02,  1.84294507e-02,
        7.40681635e-03,  3.67519446e-03, -1.52723184e-02,  1.43990701e-03,
        4.24552932e-02,  3.45296785e-03, -6.14174036e-03,  1.51932696e-02,
        9.14245564e-03, -3.98307331e-02, -2.73353886e-03,  3.08246650e-02,
        2.11118460e-02, -2.31477767e-02, -9.63426530e-02, -8.17879289e-03,
       -7.83209875e-02,  7.77560426e-03, -4.76507880e-02, -1.47334998e-02,
        7.71743208e-02,  4.59704036e-03, -5.67097999e-02, -2.02654134e-02,
        2.71433853e-02,  3.77836563e-02, -6.58802269e-03,  4.25994769e-03,
        4.81555574e-02, -1.51657537e-02,  1.24493010e-06,  4.91969362e-02,
        4.54480713e-03,  3.40875499e-02, -2.60243751e-02, -3.52002718e-02,
       -2.00636648e-02, -6.73105642e-02,  2.06303038e-02,  2.29437463e-02,
       -9.08645242e-02, -4.50733192e-02, -5.90507686e-02, -2.30297707e-02,
        1.84061509e-02, -5.30719385e-02,  2.33381335e-02,  2.10018549e-02,
       -1.39468359e-02,  

In [128]:
from sklearn.metrics.pairwise import cosine_similarity

def retrieve_top_k_chunks(question_embedding, chunks, chunk_embeddings, k=3):
    sims = cosine_similarity([question_embedding], chunk_embeddings)[0]
    top_indices = sims.argsort()[-k:][::-1]
    return [(chunks[i], i, sims[i]) for i in top_indices]


In [129]:
def trouver_page_du_chunk(chunk, ocr_texts):
    for i, page_text in enumerate(ocr_texts):
        if chunk.strip()[:50] in page_text:
            return i + 1  
    return None


In [130]:
top_chunks = retrieve_top_k_chunks(question_embeddings, chunks, chunk_embeddings, k=3)

for chunk, idx, score in top_chunks:
    page = trouver_page_du_chunk(chunk, ocr_texts)
    print(f"Chunk {idx} (score: {score:.2f}) - Page {page if page else 'inconnue'}:\n")
    print(chunk[:500] + "...")


Chunk 4 (score: 0.44) - Page 1:

Exceptions  Additionally, the following are excepted from the Proclamation:  (i) any lawful permanent resident (LPR) of the United States;  (ii) any dual national of a country designated under sections 2 and 3 of the proclamation when the individual is traveling on a passport issued by a country not so designated;  (iii) any foreign national traveling with a valid nonimmigrant visa in the following classifications: A-1, A-2, C-2, C-3, G-1, G-2, G-3, G-4, NATO-1, NATO-2, NATO-3, NATO-4, NATO-5, o...
Chunk 2 (score: 0.41) - Page 1:

The Proclamation fully or partially suspends the entry of nationals of the following 19 countries: Afghanistan,  Myanmar (Burma), Chad, Republic of The Congo, Equatorial Guinea, Eritrea, Haiti, Iran, Libya, Somalia,  Sudan, Yemen, Burundi, Cuba, Laos, Sierra Leone, Togo, Turkmenistan, and Venezuela. The Proclamation will  be implemented by CBP and partner agencies, such as U. S....
Chunk 1 (score: 0.40) - Page inconnue:

dhs. 